In [ ]:
import os
import numpy as np
import pytorch_lightning as pl
from matplotlib import pyplot as plt
from sca_utils.metrics import dumpMetrics
from attack.CNN.create_dataset import createDataset, plotTargetStatistics, printDatasetInfo
from attack.CNN.train import train
from attack.CNN.attack import guessingMetrics, getModule

In [ ]:
np.random.seed(2025)
pl.seed_everything(2025, workers=True)

## Create dataset

Create a dataset for training the CNN. The dataset is split into __training__, __validation__ and __test__ subset.

Each subset consists into three .npy files:
- _windows_: it contains the side-channel traces accordingly preprocessed.
- _targets_: it contains the target labels for trainig the CNN. Each label has 16 fields, one for each key byte.
- _meta_: it contains additional metadata to proper test the CNN, i.e., plaintext and key, for each trace.

In [ ]:
trace_folder =  "</path/to/traces/folder/>"
dataset_folder = "</path/to/out/dataset/folder/>"

In [ ]:
all_trace_files = [trace_folder + f for f in os.listdir(trace_folder)]

Function _createDataset_ has a few optional parameters:
- `window`: Choose the window to use to extract the traces. If empty the window will be the whole trace (default is []).
- `filter`: Apply a highpass filter to traces as preprocessing (default is True).
- `aggregate_n_samples`: Determine how many consecutive samples to average together as preprocessing (default is 1).
- `std`: Flag to normalize the traces feature-wise as preprocessing (default is False).

In [ ]:
createDataset(all_trace_files, dataset_folder)
plotTargetStatistics(dataset_folder)
printDatasetInfo(dataset_folder)

## Train CNN

Each CNN is configure thanks to a YALM configuration file.  
You can set different module hyper-parameters as well as the dataset, the logger, and the experiment configurations.  
Default configuration are in `configs` directory, both for Neputune logger and for the experiment. 

In [ ]:
config_folder = "attack/CNN/configs/cnn_exp_v1" #"/path/to/experiment/config/folder/"
gpu = 0 # 0 if you want to use the first GPU, 1 if you want to use the second GPU, and so on

In [ ]:
train(config_folder, gpu)

## Inference
### Compute Guessing Metrics

In [ ]:
attack_folder = "</path/to/attack/traces/folder/>"
result_folder = "</path/to/save/results/folder/>" 
SID = "<Neptun ID>"

trace_files = [attack_folder + f for f in os.listdir(attack_folder)]

Get the trained CNN starting from Neptune SID.

In [ ]:
module = getModule(SID)

Compute, save and plot metrics. In folder `result_folder` there is a .txt summary of the principal side-channel metrics.  
Function _guessingMetrics_ has a few optional parameters to __set the same as during the module training__:
- `target_byte`: Determine on which intermediate byte profile the Template (default is byte 0).
- `window`: Choose the window to use to extract the traces. If empty the window will be the whole trace (default is []).
- `filter`: Apply a highpass filter to traces as preprocessing (default is True).
- `aggregate_n_samples`: Determine how many consecutive samples to average together as preprocessing (default is 1).
- `std`: Flag to normalize the traces feature-wise as preprocessing (default is False).

In [ ]:
ge, gd, ranks = guessingMetrics(trace_files, module)
print(f"Guessing distance: {gd}, Guessing entropy: {ge}")

# Save results
dumpMetrics(result_folder, gd, ranks)

In [ ]:
fig = plt.figure(figsize=(8, 5))
plt.plot(np.mean(ranks, axis=0))
plt.grid(True)
plt.ylim(0, 180)